In [89]:
import pandas as pd
import numpy as np
import ast
from tabulate import tabulate
import json
import yaml
import matplotlib.pyplot as plt
import itertools
import dataframe_image as dfi


pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [90]:
filename = "DEFAULT_FINAL"  ## insert the name of the file used in the runner.py
df = pd.read_csv(r"../output/{filenanme}.csv")

In [91]:
# List of metric names
metric_names = [
    "lift",
    "f1",
    "mse",
    "accuracy",
    "recall",
    "precision",
    "roc_auc",
    "area_under_pr",
    "r2_score",
    "rmse",
]
dataset = df.copy()
# Create columns based on metric names
# Convert string representations to dictionaries
dataset["output_metrics"] = dataset["output_metrics"].apply(ast.literal_eval)
dataset["run_time"] = dataset["run_time"].apply(lambda x: np.round(x / 60, 2))
# Create columns based on metric names
for metric in metric_names:
    dataset[metric] = dataset["output_metrics"].apply(
        lambda x: x.get(metric) if isinstance(x, dict) else np.nan
    )
    dataset[f"{metric}_std"] = dataset[metric].apply(
        lambda x: np.std(x) if isinstance(x, list) else np.nan
    )
    dataset[metric] = dataset[metric].apply(
        lambda x: np.average(x) if isinstance(x, list) else np.nan
    )

base_cols = [
    "dataset",
    "model",
    "run_time",
    "eval_metric",
    "best_score",
    "score_std",
    "output_metrics",
]

base_cols += [i for i in metric_names]
base_cols += [i + "_std" for i in metric_names]

dataset = dataset[base_cols]

In [92]:
tomax = {
    "mse": False,
    "rmse": False,
    "accuracy": True,
    "recall": True,
    "precision": True,
    "roc_auc": True,
    "area_under_pr": True,
    "lift": True,
    "f1": True,
    "r2_score": True,
}


# Define a function to select the first row based on whether to maximize or minimize the "best_score"
def select_first_row(group):
    metric = group["eval_metric"].iloc[0]
    ascending = not tomax.get(
        metric, True
    )  # If metric not in tomax, assume True (maximize)
    return group.sort_values(by="best_score", ascending=ascending).iloc[0]


# Apply the function to each group
filtered_df = dataset.groupby(["dataset", "model", "eval_metric"]).apply(
    select_first_row
)
# Reset the index to get a new DataFrame
filtered_df = filtered_df.reset_index(drop=True)
# Find the rows that maximize the specified metric for each dataset


# Sort the DataFrame based on whether the metric is to be maximized or not
filtered_df["ascending"] = filtered_df["eval_metric"].map(
    {k: not v for k, v in tomax.items()}
)  # Create a new column for ascending order
dfmax = filtered_df.loc[filtered_df["ascending"] == False].sort_values(
    by=["dataset", "eval_metric", "best_score"], ascending=[False, False, False]
)
dfmin = filtered_df.loc[filtered_df["ascending"] == True].sort_values(
    by=["dataset", "eval_metric", "best_score"], ascending=[False, False, True]
)

best_df = pd.concat([dfmax, dfmin])
best_df.drop(columns=["ascending"], inplace=True)

best_df.head(100)

,dataset,model,run_time,eval_metric,best_score,score_std,output_metrics,lift,f1,mse,accuracy,recall,precision,roc_auc,area_under_pr,r2_score,rmse,lift_std,f1_std,mse_std,accuracy_std,recall_std,precision_std,roc_auc_std,area_under_pr_std,r2_score_std,rmse_std
109,titanic,catboost,0.53,lift,2.544501,0.089689,"{'recall': [0.7391304347826086, 0.764705882352...",2.544501,0.769090,NaN,0.836099,0.713384,0.836864,0.881115,0.857053,NaN,NaN,0.089689,0.041024,NaN,0.026399,0.050898,0.047627,0.017801,0.016332,NaN,NaN
118,titanic,tabtransformer,0.67,lift,2.544055,0.076238,"{'recall': [0.7391304347826086, 0.764705882352...",2.544055,0.740484,NaN,0.800207,0.742626,0.738722,0.843048,0.829274,NaN,NaN,0.076238,0.019533,NaN,0.015113,0.020595,0.024858,0.014490,0.020611,NaN,NaN
108,titanic,autoint,0.30,lift,2.513705,0.085630,"{'recall': [0.8115942028985508, 0.794117647058...",2.513705,0.740424,NaN,0.791243,0.777536,0.707235,0.861729,0.829969,NaN,NaN,0.085630,0.028250,NaN,0.019285,0.041999,0.020787,0.016240,0.014752,NaN,NaN
112,titanic,gandalf,0.39,lift,2.513705,0.085630,"{'recall': [0.8260869565217391, 0.779411764705...",2.513705,0.763372,NaN,0.818134,0.768627,0.759127,0.868772,0.847891,NaN,NaN,0.085630,0.050296,NaN,0.034566,0.065001,0.039928,0.023869,0.021617,NaN,NaN
119,titanic,xgb,0.01,lift,2.482910,0.119197,"{'recall': [0.7681159420289855, 0.794117647058...",2.482910,0.777888,NaN,0.838365,0.739685,0.821129,0.874329,0.847345,NaN,NaN,0.119197,0.027343,NaN,0.016595,0.040329,0.016061,0.020416,0.029427,NaN,NaN
115,titanic,resnet,2.90,lift,2.453177,0.246957,"{'recall': [0.4782608695652174, 0.617647058823...",2.453177,0.674339,NaN,0.791256,0.567732,0.839869,0.841848,0.808628,NaN,NaN,0.246957,0.053832,NaN,0.029306,0.068776,0.057776,0.028165,0.050293,NaN,NaN
114,titanic,mlp,0.01,lift,2.452390,0.143021,"{'recall': [0.6666666666666666, 0.632352941176...",2.452390,0.711148,NaN,0.797979,0.649105,0.787613,0.834473,0.799974,NaN,NaN,0.143021,0.023124,NaN,0.013769,0.034855,0.018000,0.012359,0.011375,NaN,NaN
116,titanic,s1dcnn,1.43,lift,2.298305,0.211508,"{'recall': [0.6811594202898551, 0.661764705882...",2.298305,0.723958,NaN,0.809177,0.651961,0.815512,0.837829,0.780987,NaN,NaN,0.211508,0.034296,NaN,0.023751,0.033497,0.049920,0.027274,0.051035,NaN,NaN
111,titanic,fttransformer,0.82,lift,2.205471,0.721286,"{'recall': [0.782608695652174, 0.0, 0.69117647...",2.205471,0.611493,NaN,0.777685,0.615857,0.608281,0.788943,0.755340,NaN,NaN,0.721286,0.308474,NaN,0.085766,0.311032,0.308039,0.169059,0.195174,NaN,NaN
113,titanic,gate,4.35,lift,2.175122,0.411114,"{'recall': [0.7536231884057971, 0.75, 0.705882...",2.175122,0.547951,NaN,0.731649,0.546249,0.551259,0.780041,0.724321,NaN,NaN,0.411114,0.283898,NaN,0.077389,0.286004,0.284270,0.107188,0.133788,NaN,NaN


# Check State of Run

In [93]:
with open("../configuration/experiment_config.yml", "r") as f:
    config = yaml.safe_load(f)

# Extract the necessary information from the configuration file
included_models = [i.lower() for i in config["include_models"]]
included_datasets = [i.lower() for i in config["include_datasets"]]

# Get all combinations of items from the two lists
combinations_possible = list(itertools.product(included_datasets, included_models))


# Get unique combinations based on 'Column1' and 'Column2'
existing_combinations = df[["dataset", "model"]].drop_duplicates()
# Convert the DataFrame to a list of tuples
existing_combinations = [tuple(row) for row in existing_combinations.values]

missing_combos = [i for i in combinations_possible if i not in existing_combinations]
for i in missing_combos:
    print(f"Missing Combination {i}")

In [96]:
best_df["dataset"].value_counts()

dataset
titanic          12
iris             12
housing          12
heloc            12
diabetes         12
creditcard       12
covertype        12
breastcancer     12
ageconditions    12
adult            12
Name: count, dtype: int64

# Execution Time Visualization

In [97]:
# Group by the "dataset" column and aggregate "run_time" using the sum function
hyperopt_evals = 100
num_parallel = 3
efficiency_estimate = 0.8
aggregated_df = best_df.groupby("model")["run_time"].sum().reset_index()
# Rename the aggregated column for clarity
aggregated_df = aggregated_df.rename(columns={"run_time": "sum_run_time"})
aggregated_df["total_search_hours_estimate"] = (
    aggregated_df["sum_run_time"] * 100 / 60 / 3 * 0.8
)
aggregated_df.sort_values("total_search_hours_estimate", ascending=False).head(20)

,model,sum_run_time,total_search_hours_estimate
3,fttransformer,385.05,171.133333
7,resnet,335.66,149.182222
5,gate,239.89,106.617778
4,gandalf,115.68,51.413333
10,tabtransformer,98.21,43.648889
9,tabnet,90.10,40.044444
8,s1dcnn,78.45,34.866667
2,categoryembedding,44.83,19.924444
0,autoint,34.19,15.195556
11,xgb,21.92,9.742222


In [98]:
# Convert DataFrame to ASCII table
class Format:
    end = "\033[0m"
    underline = "\033[4m"


def make_results_table(
    df, dataset_name, display_cols, metric_cols, image_name="", dpi=1200, image_path=""
):
    result_df = df.loc[df["dataset"] == dataset_name].reset_index(drop=True).copy()

    # Create a dictionary to store the indices of the rows with the highest values for each metric column
    max_indices = {}
    for metric in metric_cols:
        max_indices[metric] = result_df[metric].idxmax()

    # Modify all columns with std to include relative std
    result_df["best_score"] = result_df.apply(
        lambda row: f"{row['best_score']:.4f} ± ({row['score_std']:.4f})", axis=1
    )
    for metric in metric_cols:
        result_df[metric] = result_df.apply(
            lambda row: f"{row[metric]:.4f} ± ({row[metric+'_std']:.4f})", axis=1
        )
        # Drop the corresponding std column
        result_df.drop(columns=[metric + "_std"], inplace=True)

    result_df[display_cols].to_csv(rf"{image_path}/{image_name}.csv")

    return result_df[display_cols]


# Define a custom styling function
def highlight_max_row(s):
    is_max = s == s.max()
    return ["background-color: green" if v else "" for v in is_max]


def highlight_min_row(s):
    is_max = s == s.min()
    return ["background-color: green" if v else "" for v in is_max]


folder = "default"

# Housing


In [79]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = [
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
dataset_name = "titanic"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,lift,f1,recall,precision,area_under_pr
0,catboost,0.8361 ± (0.0264),0.8811 ± (0.0178),2.5445 ± (0.0897),0.7691 ± (0.0410),0.7134 ± (0.0509),0.8369 ± (0.0476),0.8571 ± (0.0163)
1,tabtransformer,0.8002 ± (0.0151),0.8430 ± (0.0145),2.5441 ± (0.0762),0.7405 ± (0.0195),0.7426 ± (0.0206),0.7387 ± (0.0249),0.8293 ± (0.0206)
2,autoint,0.7912 ± (0.0193),0.8617 ± (0.0162),2.5137 ± (0.0856),0.7404 ± (0.0283),0.7775 ± (0.0420),0.7072 ± (0.0208),0.8300 ± (0.0148)
3,gandalf,0.8181 ± (0.0346),0.8688 ± (0.0239),2.5137 ± (0.0856),0.7634 ± (0.0503),0.7686 ± (0.0650),0.7591 ± (0.0399),0.8479 ± (0.0216)
4,xgb,0.8384 ± (0.0166),0.8743 ± (0.0204),2.4829 ± (0.1192),0.7779 ± (0.0273),0.7397 ± (0.0403),0.8211 ± (0.0161),0.8473 ± (0.0294)
5,resnet,0.7913 ± (0.0293),0.8418 ± (0.0282),2.4532 ± (0.2470),0.6743 ± (0.0538),0.5677 ± (0.0688),0.8399 ± (0.0578),0.8086 ± (0.0503)
6,mlp,0.7980 ± (0.0138),0.8345 ± (0.0124),2.4524 ± (0.1430),0.7111 ± (0.0231),0.6491 ± (0.0349),0.7876 ± (0.0180),0.8000 ± (0.0114)
7,s1dcnn,0.8092 ± (0.0238),0.8378 ± (0.0273),2.2983 ± (0.2115),0.7240 ± (0.0343),0.6520 ± (0.0335),0.8155 ± (0.0499),0.7810 ± (0.0510)
8,fttransformer,0.7777 ± (0.0858),0.7889 ± (0.1691),2.2055 ± (0.7213),0.6115 ± (0.3085),0.6159 ± (0.3110),0.6083 ± (0.3080),0.7553 ± (0.1952)
9,gate,0.7316 ± (0.0774),0.7800 ± (0.1072),2.1751 ± (0.4111),0.5480 ± (0.2839),0.5462 ± (0.2860),0.5513 ± (0.2843),0.7243 ± (0.1338)


In [80]:
display_cols = ["model", "mse", "r2_score", "rmse"]
metric_cols = ["mse", "r2_score", "rmse"]
dataset_name = "housing"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=["r2_score"], axis=0).apply(
    highlight_min_row, subset=["mse", "rmse"], axis=0
)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,mse,r2_score,rmse
0,catboost,0.2058 ± (0.0106),0.8454 ± (0.0085),0.4535 ± (0.0118)
1,xgb,0.2135 ± (0.0069),0.8396 ± (0.0063),0.4620 ± (0.0075)
2,mlp,0.2976 ± (0.0169),0.7765 ± (0.0112),0.5453 ± (0.0155)
3,s1dcnn,0.3009 ± (0.0130),0.7741 ± (0.0066),0.5484 ± (0.0119)
4,resnet,0.3068 ± (0.0087),0.7695 ± (0.0073),0.5538 ± (0.0078)
5,gandalf,0.3402 ± (0.0245),0.7444 ± (0.0195),0.5829 ± (0.0215)
6,categoryembedding,0.3521 ± (0.0110),0.7355 ± (0.0100),0.5933 ± (0.0093)
7,gate,0.3851 ± (0.0268),0.7107 ± (0.0199),0.6202 ± (0.0221)
8,fttransformer,0.3901 ± (0.0234),0.7069 ± (0.0171),0.6243 ± (0.0189)
9,autoint,0.4057 ± (0.0183),0.6953 ± (0.0114),0.6368 ± (0.0143)


In [99]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = [
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
dataset_name = "heloc"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,lift,f1,recall,precision,area_under_pr
0,catboost,0.7231 ± (0.0075),0.7980 ± (0.0065),1.7197 ± (0.0286),0.7436 ± (0.0049),0.7690 ± (0.0053),0.7199 ± (0.0107),0.7953 ± (0.0090)
1,fttransformer,0.7212 ± (0.0068),0.7917 ± (0.0025),1.7124 ± (0.0235),0.7364 ± (0.0045),0.7459 ± (0.0100),0.7273 ± (0.0116),0.7908 ± (0.0059)
2,mlp,0.7179 ± (0.0065),0.7887 ± (0.0067),1.6959 ± (0.0301),0.7370 ± (0.0051),0.7575 ± (0.0089),0.7178 ± (0.0092),0.7865 ± (0.0083)
3,gandalf,0.7210 ± (0.0038),0.7929 ± (0.0049),1.6941 ± (0.0310),0.7360 ± (0.0099),0.7463 ± (0.0329),0.7275 ± (0.0143),0.7896 ± (0.0095)
4,gate,0.7213 ± (0.0076),0.7924 ± (0.0046),1.6904 ± (0.0456),0.7413 ± (0.0058),0.7648 ± (0.0036),0.7192 ± (0.0089),0.7900 ± (0.0073)
5,xgb,0.7211 ± (0.0085),0.7897 ± (0.0089),1.6886 ± (0.0147),0.7434 ± (0.0065),0.7741 ± (0.0095),0.7153 ± (0.0112),0.7863 ± (0.0077)
6,categoryembedding,0.7178 ± (0.0063),0.7860 ± (0.0045),1.6886 ± (0.0335),0.7293 ± (0.0110),0.7293 ± (0.0253),0.7299 ± (0.0047),0.7850 ± (0.0054)
7,tabtransformer,0.7141 ± (0.0059),0.7789 ± (0.0063),1.6867 ± (0.0344),0.7242 ± (0.0120),0.7201 ± (0.0295),0.7293 ± (0.0086),0.7754 ± (0.0072)
8,resnet,0.7127 ± (0.0069),0.7838 ± (0.0054),1.6776 ± (0.0367),0.7389 ± (0.0054),0.7791 ± (0.0190),0.7032 ± (0.0129),0.7814 ± (0.0062)
9,s1dcnn,0.6955 ± (0.0084),0.7818 ± (0.0072),1.6556 ± (0.0405),0.6665 ± (0.0197),0.5845 ± (0.0348),0.7775 ± (0.0104),0.7765 ± (0.0107)


In [82]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = [
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
dataset_name = "diabetes"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,lift,f1,recall,precision,area_under_pr
0,xgb,0.8886 ± (0.0004),0.6786 ± (0.0047),2.4711 ± (0.0670),0.0319 ± (0.0087),0.0165 ± (0.0046),0.5209 ± (0.0533),0.2308 ± (0.0062)
1,catboost,0.8886 ± (0.0003),0.6811 ± (0.0059),2.4535 ± (0.0861),0.0252 ± (0.0101),0.0129 ± (0.0053),0.5203 ± (0.0894),0.2342 ± (0.0093)
2,gate,0.6240 ± (0.0335),0.6780 ± (0.0058),2.4429 ± (0.0398),0.2712 ± (0.0069),0.6251 ± (0.0356),0.1735 ± (0.0084),0.2246 ± (0.0102)
3,s1dcnn,0.7925 ± (0.0093),0.6533 ± (0.0095),2.2968 ± (0.0861),0.2636 ± (0.0061),0.3328 ± (0.0154),0.2187 ± (0.0081),0.1960 ± (0.0073)
4,gandalf,0.6620 ± (0.0588),0.6593 ± (0.0326),2.2818 ± (0.2502),0.2614 ± (0.0228),0.5431 ± (0.1283),0.1751 ± (0.0127),0.2087 ± (0.0228)
5,mlp,0.8877 ± (0.0007),0.6522 ± (0.0120),2.2677 ± (0.0933),0.0377 ± (0.0066),0.0197 ± (0.0035),0.4365 ± (0.0589),0.2054 ± (0.0108)
6,tabtransformer,0.6564 ± (0.0480),0.6594 ± (0.0231),2.2448 ± (0.2592),0.2637 ± (0.0131),0.5522 ± (0.0817),0.1748 ± (0.0104),0.2068 ± (0.0201)
7,resnet,0.8884 ± (0.0000),0.6520 ± (0.0086),2.2263 ± (0.1146),0.0000 ± (0.0000),0.0000 ± (0.0000),0.0000 ± (0.0000),0.2056 ± (0.0113)
8,fttransformer,0.5316 ± (0.2114),0.6396 ± (0.0699),2.1083 ± (0.5315),0.2570 ± (0.0284),0.6835 ± (0.1611),0.1623 ± (0.0260),0.1983 ± (0.0439)
9,tabnet,0.6855 ± (0.0829),0.6247 ± (0.0260),2.0863 ± (0.1967),0.2474 ± (0.0118),0.4664 ± (0.1373),0.1784 ± (0.0274),0.1802 ± (0.0176)


In [83]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = [
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
dataset_name = "creditcard"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,lift,f1,recall,precision,area_under_pr
0,catboost,0.9996 ± (0.0001),0.9854 ± (0.0085),9.6345 ± (0.1520),0.8694 ± (0.0226),0.7988 ± (0.0239),0.9540 ± (0.0259),0.8592 ± (0.0232)
1,xgb,0.9996 ± (0.0001),0.9791 ± (0.0068),9.4109 ± (0.1620),0.8649 ± (0.0212),0.7927 ± (0.0164),0.9519 ± (0.0326),0.8560 ± (0.0222)
2,mlp,0.9995 ± (0.0001),0.9699 ± (0.0069),9.3907 ± (0.1699),0.8297 ± (0.0177),0.7723 ± (0.0233),0.8977 ± (0.0338),0.8259 ± (0.0281)
3,s1dcnn,0.9744 ± (0.0131),0.9420 ± (0.0178),9.2476 ± (0.2300),0.1335 ± (0.0612),0.8901 ± (0.0315),0.0737 ± (0.0371),0.1142 ± (0.0496)
4,resnet,0.9983 ± (0.0000),0.9617 ± (0.0099),9.1049 ± (0.2386),0.0000 ± (0.0000),0.0000 ± (0.0000),0.0000 ± (0.0000),0.6970 ± (0.0645)
5,gate,0.9915 ± (0.0054),0.8990 ± (0.1320),8.0433 ± (1.8891),0.2035 ± (0.1309),0.6760 ± (0.3438),0.1227 ± (0.0840),0.5268 ± (0.2571)
6,gandalf,0.9564 ± (0.0434),0.8090 ± (0.3428),7.5726 ± (3.6872),0.1023 ± (0.0548),0.7266 ± (0.3334),0.0552 ± (0.0301),0.5858 ± (0.2928)
7,fttransformer,0.9891 ± (0.0108),0.6718 ± (0.3730),5.7375 ± (4.4769),0.1534 ± (0.1947),0.5271 ± (0.4310),0.1022 ± (0.1430),0.3966 ± (0.3248)
8,autoint,0.9235 ± (0.0545),0.7867 ± (0.1376),4.7979 ± (2.8635),0.0748 ± (0.0818),0.5451 ± (0.3132),0.0418 ± (0.0466),0.1525 ± (0.2734)
9,categoryembedding,0.9145 ± (0.1457),0.6400 ± (0.1745),3.5115 ± (3.1384),0.0125 ± (0.0215),0.3208 ± (0.3929),0.0064 ± (0.0111),0.1182 ± (0.2116)


In [84]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = ["accuracy", "roc_auc", "f1", "recall", "precision", "area_under_pr"]
dataset_name = "adult"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,f1,recall,precision,area_under_pr
0,catboost,0.8742 ± (0.0030),0.9297 ± (0.0012),0.7148 ± (0.0063),0.6545 ± (0.0082),0.7875 ± (0.0107),0.8321 ± (0.0036)
1,xgb,0.8733 ± (0.0031),0.9285 ± (0.0017),0.7135 ± (0.0075),0.6550 ± (0.0096),0.7836 ± (0.0085),0.8300 ± (0.0047)
2,categoryembedding,0.8018 ± (0.0177),0.9077 ± (0.0037),0.6782 ± (0.0141),0.8649 ± (0.0241),0.5590 ± (0.0278),0.7620 ± (0.0124)
3,mlp,0.8519 ± (0.0022),0.9074 ± (0.0034),0.6647 ± (0.0057),0.6097 ± (0.0148),0.7313 ± (0.0124),0.7679 ± (0.0100)
4,gandalf,0.8049 ± (0.0150),0.9073 ± (0.0041),0.6784 ± (0.0080),0.8541 ± (0.0505),0.5655 ± (0.0299),0.7650 ± (0.0100)
5,resnet,0.8298 ± (0.0079),0.9045 ± (0.0038),0.4941 ± (0.0465),0.3482 ± (0.0505),0.8675 ± (0.0229),0.7661 ± (0.0094)
6,tabtransformer,0.7856 ± (0.0310),0.8879 ± (0.0345),0.6559 ± (0.0410),0.8452 ± (0.0386),0.5363 ± (0.0403),0.7195 ± (0.0788)
7,s1dcnn,0.8173 ± (0.0110),0.8740 ± (0.0125),0.6660 ± (0.0072),0.7563 ± (0.0364),0.5972 ± (0.0264),0.6690 ± (0.0108)
8,gate,0.7920 ± (0.0255),0.8731 ± (0.0535),0.6212 ± (0.0905),0.7574 ± (0.2179),0.5719 ± (0.0656),0.7059 ± (0.0835)
9,fttransformer,0.5883 ± (0.2838),0.8243 ± (0.1469),0.5706 ± (0.1491),0.9040 ± (0.0807),0.4502 ± (0.1718),0.6509 ± (0.1963)


In [85]:
display_cols = ["model", "accuracy", "f1"]
metric_cols = [
    "accuracy",
    "f1",
]
dataset_name = "iris"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,f1
0,autoint,0.9800 ± (0.0267),0.9800 ± (0.0267)
1,fttransformer,0.9800 ± (0.0267),0.9798 ± (0.0269)
2,catboost,0.9600 ± (0.0249),0.9598 ± (0.0251)
3,tabtransformer,0.9600 ± (0.0490),0.9599 ± (0.0491)
4,xgb,0.9600 ± (0.0249),0.9598 ± (0.0251)
5,gandalf,0.9467 ± (0.0542),0.9463 ± (0.0546)
6,s1dcnn,0.9333 ± (0.0471),0.9330 ± (0.0470)
7,gate,0.9133 ± (0.1258),0.8911 ± (0.1696)
8,resnet,0.9133 ± (0.0806),0.9127 ± (0.0808)
9,categoryembedding,0.9067 ± (0.1062),0.9014 ± (0.1148)


In [86]:
display_cols = ["model", "accuracy", "f1"]
metric_cols = ["accuracy", "f1"]
dataset_name = "covertype"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,f1
0,resnet,0.9206 ± (0.0078),0.9226 ± (0.0075)
1,catboost,0.8925 ± (0.0008),0.8919 ± (0.0008)
2,gandalf,0.8790 ± (0.0175),0.8824 ± (0.0166)
3,s1dcnn,0.8740 ± (0.0511),0.8784 ± (0.0478)
4,xgb,0.8690 ± (0.0004),0.8684 ± (0.0005)
5,mlp,0.8581 ± (0.0031),0.8564 ± (0.0033)
6,categoryembedding,0.7420 ± (0.0535),0.7546 ± (0.0487)
7,gate,0.6800 ± (0.0349),0.6979 ± (0.0306)
8,tabtransformer,0.5921 ± (0.0283),0.6222 ± (0.0220)
9,tabnet,0.5911 ± (0.1199),0.5734 ± (0.1925)


In [87]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = [
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
dataset_name = "breastcancer"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.sort_values("f1", ascending=False).style.apply(
    highlight_max_row, subset=metric_cols, axis=0
)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,lift,f1,recall,precision,area_under_pr
4,gate,0.9789 ± (0.0089),0.9961 ± (0.0059),1.5939 ± (0.0100),0.9834 ± (0.0069),0.9888 ± (0.0104),0.9783 ± (0.0184),0.9973 ± (0.0042)
3,gandalf,0.9754 ± (0.0129),0.9954 ± (0.0070),1.5939 ± (0.0100),0.9805 ± (0.0103),0.9832 ± (0.0138),0.9780 ± (0.0162),0.9964 ± (0.0059)
2,fttransformer,0.9754 ± (0.0065),0.9957 ± (0.0061),1.5939 ± (0.0100),0.9804 ± (0.0051),0.9804 ± (0.0067),0.9804 ± (0.0067),0.9970 ± (0.0046)
1,catboost,0.9737 ± (0.0078),0.9935 ± (0.0067),1.5939 ± (0.0100),0.9791 ± (0.0061),0.9832 ± (0.0105),0.9753 ± (0.0158),0.9949 ± (0.0058)
8,tabtransformer,0.9649 ± (0.0136),0.9953 ± (0.0043),1.5939 ± (0.0100),0.9720 ± (0.0108),0.9721 ± (0.0197),0.9725 ± (0.0191),0.9970 ± (0.0028)
9,xgb,0.9613 ± (0.0212),0.9935 ± (0.0054),1.5939 ± (0.0100),0.9697 ± (0.0162),0.9804 ± (0.0142),0.9600 ± (0.0321),0.9958 ± (0.0037)
6,resnet,0.9490 ± (0.0172),0.9935 ± (0.0094),1.5939 ± (0.0100),0.9600 ± (0.0122),0.9665 ± (0.0287),0.9559 ± (0.0419),0.9955 ± (0.0068)
5,mlp,0.9367 ± (0.0311),0.9835 ± (0.0086),1.5939 ± (0.0100),0.9486 ± (0.0268),0.9413 ± (0.0507),0.9575 ± (0.0115),0.9892 ± (0.0053)
7,s1dcnn,0.8085 ± (0.1239),0.9909 ± (0.0048),1.5939 ± (0.0100),0.8668 ± (0.0728),0.9382 ± (0.0928),0.8317 ± (0.1500),0.9936 ± (0.0039)
10,categoryembedding,0.8631 ± (0.2215),0.9594 ± (0.0728),1.5063 ± (0.1695),0.8183 ± (0.3219),0.8030 ± (0.3524),0.9391 ± (0.0827),0.9674 ± (0.0593)


In [88]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = [
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
dataset_name = "ageconditions"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.sort_values("f1", ascending=False).style.apply(
    highlight_max_row, subset=metric_cols, axis=0
)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,lift,f1,recall,precision,area_under_pr
2,s1dcnn,0.9473 ± (0.0106),0.9606 ± (0.0134),5.0003 ± (0.5966),0.8528 ± (0.0292),0.8705 ± (0.0372),0.8369 ± (0.0378),0.8348 ± (0.0594)
0,catboost,0.9416 ± (0.0209),0.9670 ± (0.0245),5.2348 ± (0.4964),0.8162 ± (0.0640),0.7407 ± (0.0630),0.9140 ± (0.0912),0.8886 ± (0.0808)
1,xgb,0.9157 ± (0.0217),0.9572 ± (0.0320),5.2348 ± (0.2364),0.7329 ± (0.0561),0.6571 ± (0.0641),0.8468 ± (0.1150),0.8776 ± (0.0628)
3,gandalf,0.8850 ± (0.0543),0.9474 ± (0.0372),4.7674 ± (0.5775),0.7283 ± (0.0981),0.8429 ± (0.0857),0.6537 ± (0.1291),0.8256 ± (0.0783)
5,gate,0.8688 ± (0.0400),0.9120 ± (0.0325),4.6609 ± (0.5058),0.6835 ± (0.0546),0.7870 ± (0.0224),0.6106 ± (0.0829),0.7312 ± (0.0775)
4,autoint,0.8785 ± (0.0177),0.8626 ± (0.0717),4.6617 ± (0.4189),0.6377 ± (0.1378),0.6879 ± (0.2380),0.7070 ± (0.1571),0.7265 ± (0.0928)
7,tabtransformer,0.7858 ± (0.1616),0.8201 ± (0.1605),3.8925 ± (1.2428),0.5907 ± (0.1706),0.7403 ± (0.1392),0.5132 ± (0.1821),0.6357 ± (0.2167)
8,categoryembedding,0.8509 ± (0.0420),0.7861 ± (0.2041),3.8852 ± (1.4910),0.5458 ± (0.2425),0.6074 ± (0.2805),0.5261 ± (0.1969),0.6478 ± (0.2551)
6,mlp,0.8734 ± (0.0513),0.8645 ± (0.0338),3.9998 ± (0.9324),0.5349 ± (0.2398),0.4623 ± (0.2329),0.6529 ± (0.2183),0.6482 ± (0.1373)
11,tabnet,0.3976 ± (0.2265),0.5749 ± (0.0990),1.5213 ± (0.6885),0.2953 ± (0.0397),0.7069 ± (0.2852),0.2241 ± (0.1032),0.2591 ± (0.0739)
